# Using the PyLabRobot Simulator

This notebook servers as an introduction to the PyLabRobot simulator. It provides a very quick overview of the simulator and PLR in general. More documentation is available [on the docs website](https://docs.pylabrobot.org).

To  get started, import LiquidHandler and SimulatorBackend. The LiquidHandler (or lh for short), will be our interface to all liquid handling functionality in PLR. The SimulatorBackend is one of many available _backends_, classes that handle communication, but a very special one.

In [ ]:
from pylabrobot.liquid_handling import LiquidHandler
from pylabrobot.liquid_handling.backends.simulation.simulator_backend import SimulatorBackend

Next, import a few of the VENUS resources, which we'll use to create a deck.

In [ ]:
from pylabrobot.liquid_handling.resources import (
    TIP_CAR_480_A00,
    PLT_CAR_L5AC_A00,
    Cos_96_DW_1mL,
    STF_L
)

Then instantiate our classes, passing the backend as an argument to LH.

**note: it is very important that you keep the hosts in the simulator backend while using simulator.pylaborobot.org**

In [ ]:
sb = SimulatorBackend(ws_host="0.0.0.0", fs_host="0.0.0.0")
lh = LiquidHandler(backend=sb)

When you're ready to start using LH, call `.setup()`.

In [ ]:
lh.setup()

This convenience method waits until a connection with the simulator is established.

In [ ]:
sb.wait_for_connection()

You should now see the simulator on the right hand side of your screen.

## Defining a deck layout

With a physical deck, you'd set up your resources before telling PLR where they are. With the simulator however, just the second step is required. The simulator will automatically display them.

In [ ]:
tip_car = TIP_CAR_480_A00(name='tip carrier')
for i in range(2):
    tip_car[i] = STF_L(name=f'tip_{i}')

In [ ]:
tip_car[3] = STF_L(name=f'tip_{3}')

In [ ]:
lh.assign_resource(tip_car, rails=1, replace=True)

You should now see a tip carrier appear in the simulator window.

Next, let's create a plate carrier.

In [ ]:
plt_car = PLT_CAR_L5AC_A00(name='pre')
for i in range(5):
    plt_car[i] = Cos_96_DW_1mL(name=f'plate_{i}')

In [ ]:
lh.assign_resource(plt_car, rails=9, replace=True)

### Further configuring the deck

Just like in reality, only assigning where your resources are is not enough to start liquid handling. You also need supplies like tips and liquid. The simulator provides methods to create these virtually. This allows it to perform "reality checks", and raise errors, if some part of the liquid handling would not be physically possible. Since this simulator specific functionality, the methods are part of `sb` (the simulator backend variable).

To start, let's fill the first tip resource with tips.

In [ ]:
tips = lh.get_resource('tip_0') # use the name of the resource we just defined
sb.fill_tips(tips)

As you can see, the circles in the simulator are now filled to indicate tips are present.

Adding liquid to wells is done in a similar way. Below we alternate the volume in the wells in the columns with 100 and 800 microliters.

In [ ]:
plate = lh.get_resource('plate_0')
sb.adjust_well_volume(plate, pattern=[[100, 800]*6]*8)

## Liquid handling

Now that we have tips and liquid set up, we are ready to do some virtual liquid handling. This is done through `lh`, the generic liquid handler front end. This allows us to easily reuse our liquid handling code with physical liquid handlers in the future.

Start by picking up a few tips. We can use (transposed) MS Excel type notation to easily select entire columns. Just like with a robot, this operation takes a while (2 seconds). However, unlike physical reality, you can use the settings button on to edit these durations.

In [ ]:
lh.pickup_tips(tips["A1:H1"])

Next, let's aspirate some liquid from the plate and move it to have an equal volume of liquid in the first two columns. Notice how the color of the wells reflects the amount of liquid in that well.

In [ ]:
vols = [350]*8
lh.aspirate(plate["A2:H2"], vols=vols)

In [ ]:
lh.dispense(plate["A1:H1"], vols=vols)

Lastly, put the tips back in another tip resource.

In [ ]:
other_tips = lh.get_resource("tip_1")
lh.discard_tips(other_tips["A1:H1"])